# Fine-tune (Jittor SBERT)

This notebook runs NLI fine-tuning with STS eval using the Jittor pipeline.

In [ ]:
# Adjust paths before running
BASE_MODEL = "bert-base-uncased"
DATA_DIR = "./data"
OUTPUT_DIR = "./checkpoints/demo_nli"


In [ ]:
# Run NLI fine-tuning (Jittor)
!python training/nli/train_nli.py \
    {BASE_MODEL} \
    --data_dir {DATA_DIR} \
    --output_dir {OUTPUT_DIR} \
    --epochs 1 \
    --batch_size 16 \
    --eval_steps 1000 \
    --use_cuda


Optional: change pooling/loss/ablation via CLI flags.